# Anthropic Claude

In the v0.2.30 release of AutoGen we support Anthropic Client.

Claude is a family of large language models developed by Anthropic and designed to revolutionize the way you interact with AI. Claude excels at a wide variety of tasks involving language, reasoning, analysis, coding, and more. The models are highly capable, easy to use, and can be customized to suit your needs.

In this notebook, we demonstrate how to use Anthropic Claude model for AgentChat in AutoGen.

## Features

Additionally, this client class provides support for function/tool calling and will track token usage and cost correctly as per Anthropic's API costs (as of June 2024).

## Requirements
To use Anthropic Claude with AutoGen, first you need to install the `pyautogen["anthropic]` package.

To try out the function call feature of Claude model, you need to install `anthropic>=0.23.1`.


In [ ]:
# !pip install pyautogen
!pip install pyautogen["anthropic"]

## Set the config for the Anthropic API

You can add any parameters that are needed for the custom model loading in the same configuration list.

It is important to add the `api_type` field and set it to a string that corresponds to the client type used: `anthropic`.

Example:
```
[
    {
        "model": "claude-3-sonnet-20240229",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
        "temperature": 0.5,
        "top_p": 0.2, # Note: It is recommended to set temperature or top_p but not both.
        "max_tokens": 10000,
    },
    {
        "model":"claude-3-opus-20240229",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.0",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.1",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-3.0-haiku",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
]
```

### Alternative

As an alternative to the api_key key and value in the config, you can set the environment variable `ANTHROPIC_API_KEY` to your Anthropic API key.

Linux/Mac:
```
export ANTHROPIC_API_KEY="your Anthropic API key here"
```
Windows:
```
set ANTHROPIC_API_KEY=your_anthropic_api_key_here
```

In [ ]:
import os

from typing_extensions import Annotated

import autogen

config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-sonnet-20240229",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

# Coding Example with Two Agent

## Construct Agents

Construct a simple conversation between a User proxy and an ConversableAgent based on Claude-3 model.


In [ ]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_claude,
    },
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=1,
)

## Initiate Chat

In [ ]:
user_proxy.initiate_chat(
    assistant, message="Write a python program to print the first 10 numbers of the Fibonacci sequence."
)

# Function Call in Latest Anthropic API 
Anthropic just announced that tool use is now supported in the Anthropic API. To use this feature, please install `anthropic>=0.23.1`.

## Register the function

In [ ]:
@user_proxy.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@assistant.register_for_llm(
    name="get_weather", description="Get the current weather in a given location."
)  # Decorator factory for registering a function to be used by an agent
def preprocess(location: Annotated[str, "The city and state, e.g. Toronto, ON."]) -> str:
    return "Absolutely cloudy and rainy"

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="What's the weather in Toronto?",
)